In [12]:
import numpy as np
import pandas as pd
import csv_figures
from csv_figures import add_time_col
from csv_figures import annual_Norm
import matplotlib.pyplot as plt
import requests
from matplotlib.widgets import CheckButtons

In [13]:
manley = pd.read_excel('../../Cleaned_data/RuralAk/RuralAK_Manley_Tribal_Council_cleaned.xlsx')
manley_new = add_time_col(manley)
manley_annual_values,manley_month = annual_Norm(manley_new)

/Users/guge/Desktop/acep-solar/new/ACEP_solar/GEGU/figures/csv_figures.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe['Date'][i] = str(dataframe['Year'][i]) + '-' + str(dataframe['Month'][i])


In [14]:
manley

,Energy,DC Capacity,Location,Date
0,52.58,6.0,Manley Tribal Council,2014-07-01
1,189.10,NaN,NaN,2014-08-01
2,319.42,NaN,NaN,2014-09-01
3,219.69,NaN,NaN,2014-10-01
4,149.77,NaN,NaN,2014-11-01
5,10.96,NaN,NaN,2014-12-01
6,91.24,NaN,NaN,2015-01-01
7,281.80,NaN,NaN,2015-02-01
8,597.11,NaN,NaN,2015-03-01
9,731.68,NaN,NaN,2015-04-01


In [15]:
new_manley = pd.DataFrame({'Date':manley_month,'manley':manley_annual_values})

In [16]:
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 6, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 65.00, "lon": -150.63, "dataset": 'tmy2'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY2 = pd.DataFrame(data = json_response['outputs'])
# Get the data from the PV Watts --TMY2
list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 6, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": 50, "azimuth": 180, "lat": 65.00, "lon": -150.63, "dataset": 'tmy3'}
json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
TMY3 = pd.DataFrame(data = json_response['outputs'])

In [17]:
TMY2_low = TMY2['ac_annual'][2]*0.95/6
TMY2_high = TMY2['ac_annual'][2]*1.05/6

TMY3_low = TMY3['ac_annual'][2]*0.95/6
TMY3_high = TMY3['ac_annual'][2]*1.05/6

new_manley['TMY2_low'] = TMY2_low
new_manley['TMY2_high'] = TMY2_high
new_manley['TMY3_low'] = TMY3_low
new_manley['TMY3_high'] = TMY3_high
new_manley

,Date,manley,TMY2_low,TMY2_high,TMY3_low,TMY3_high
0,2015-07-01,639.998333,970.802047,1072.991736,781.373918,863.623804
1,2015-08-01,693.703333,970.802047,1072.991736,781.373918,863.623804
2,2015-09-01,720.001667,970.802047,1072.991736,781.373918,863.623804
3,2015-10-01,721.575000,970.802047,1072.991736,781.373918,863.623804
4,2015-11-01,725.058333,970.802047,1072.991736,781.373918,863.623804
5,2015-12-01,703.338333,970.802047,1072.991736,781.373918,863.623804


In [19]:
%matplotlib qt5
fig, ax = plt.subplots(figsize = (15, 8))
x = new_manley['Date']


y_manley = new_manley['manley']
l0, = ax.plot(x ,y_manley,label='manley')

y_TMY2_low = new_manley['TMY2_low']
l1, = ax.plot(x,y_TMY2_low,'-.',label = 'TMY2_low')

y_TMY2_high = new_manley['TMY2_high']
l2, = ax.plot(x,y_TMY2_high,'-.',label = 'TMY2_high')

y_TMY3_low = new_manley['TMY3_low']
l3, = ax.plot(x,y_TMY3_low,'-.',label = 'TMY3_low')

y_TMY3_high = new_manley['TMY3_high']
l4, = ax.plot(x,y_TMY3_high,'-.',label = 'TMY3_high')

ax.set_title("Rolling 12-Month Average of Produced Power, Over Time", size = 18)
ax.set_xlabel("Month", size = 14)
ax.set_ylabel("AC Power Produced (kWh) per DC Power Installed (kW)", size = 14)
ax.legend()


plt.subplots_adjust(left=0.2)
lines = [l0, l1, l2, l3, l4]

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.005, 0.48, 0.15, 0.4])
labels = [str(line.get_label()) for line in lines]
visibility = [line.get_visible() for line in lines]
check = CheckButtons(rax, labels, visibility)


def func(label):
    index = labels.index(label)
    lines[index].set_visible(not lines[index].get_visible())
    plt.draw()

check.on_clicked(func)


# Some 

plt.show()

#Ok,this seems to be fine,maybe we will talk about it after.